## Initial setup

In [14]:
import importlib
import brikasutils as bu
importlib.reload(bu)
import fb_msg_reader as fb
importlib.reload(fb)
import shared_utils as utils
from shared_utils import systemMsg, userMsg, assistantMsg
importlib.reload(utils)
import survey
importlib.reload(survey)
import persona
importlib.reload(persona)

import ollama
import numpy as np
from numpy.linalg import norm
import pandas as pd
import os
import re
import time
import json
from typing import List
from openai import OpenAI

## Persona

In [63]:
et = persona.PersonaEncoder()

# ==== FB messages ====
et.parse_fb_messages(["data-raw/1_airidas.json"], "airidas")
et.parse_fb_messages(["data-raw/2_christian.json"], "christian")
et.parse_fb_messages(["data-raw/1_nikolay.json"], "nikolay")
et.parse_fb_messages(["data-raw/2_mathis.json"], "mathis")
et.parse_fb_messages(["data-raw/2_jacob.json"], "jacob")
et.parse_fb_messages(["data-raw/2_chris.json"], "chris")
et.parse_fb_messages(["data-raw/3_aziz.json"], "aziz")
et.parse_fb_messages(["data-raw/3_daniela.json"], "daniela")
et.parse_fb_messages(["data-raw/3_mihi.json"], "mihi")
et.parse_fb_messages(["data-raw/3_viktoria.json"], "viktoria")
et.parse_fb_messages(["data-raw/4_diba.json"], "diba")
et.parse_fb_messages(["data-raw/6_filip.json"], "filip")
#et.parse_wa_messages(texts_with_rebecca, "rebecca")
# texts_with_others_dict = {
#     "rebecca": ["data-raw/messages_1000.json"],
# }
# for name, texts in texts_with_others_dict.items():
#     et.parse_fb_messages(texts, name)

# Regex cleaning
et.filter_chats_empty()
et.filter_chats_regex(utils.BLACKLIST_CHAT_REGEX_FILTERS)

# Compress names
for nameid, chat in et.chats.items():
    for msg in chat:  
        msg.sender = "Persona" if msg.sender == "Elias Salvador Smidt Torjani"  else "Friend"

# Start all chats from 2/3rds
# for name, chat in et.chats.items():
#     et.chats[name] = chat[int(len(chat)/3 * 2):]
# Select the final modules
et.select_chat_limited_by_tokens("airidas", 10000)
et.select_chat_limited_by_tokens("christian", 10000)
et.select_chat_full("nikolay")
et.select_chat_full("mathis") 
et.select_chat_full("daniela")
et.select_chat_full("diba")
et.select_chat_full("aziz")
et.select_chat_full("jacob")  
et.select_chat_full("chris")
et.select_chat_full("filip")
et.select_chat_full("mihi")
et.select_chat_full("viktoria")
# for name in texts_with_others_dict.keys():
#     ab.select_chat_full(name)

# save
big_module=et.output()
bu.quickTXT(big_module, filename=f"data/big_module_{bu.get_timestamp()}")

# stats
token_counts = et.count_all_selected_chat_tokens() # token_counts used later for statistics
print(f"Combined tokens: {sum(token_counts.values())}")
# utils.count_tokens(big_module) 
# or list(et.selectedChats.keys()) --> et.count_chat_tokens("{friend}")
# et.selectedChats["{friend}"][:5]

Read 1916 messages from 1 files. Failed to read 0 messages.
Messages ranged from 2021-09-13 to 2024-03-06
Messages saved to self.chats['airidas']
Read 618 messages from 1 files. Failed to read 0 messages.
Messages ranged from 2021-09-10 to 2024-03-03
Messages saved to self.chats['christian']
Read 297 messages from 1 files. Failed to read 0 messages.
Messages ranged from 2018-07-25 to 2024-01-01
Messages saved to self.chats['nikolay']
Read 144 messages from 1 files. Failed to read 0 messages.
Messages ranged from 2021-03-28 to 2021-12-30
Messages saved to self.chats['mathis']
Read 104 messages from 1 files. Failed to read 0 messages.
Messages ranged from 2021-08-25 to 2024-03-05
Messages saved to self.chats['jacob']
Read 159 messages from 1 files. Failed to read 0 messages.
Messages ranged from 2021-10-12 to 2023-04-30
Messages saved to self.chats['chris']
Read 161 messages from 1 files. Failed to read 0 messages.
Messages ranged from 2021-03-28 to 2021-06-06
Messages saved to self.chat

## Survey

In [91]:
surv = survey.PersonalitySurvey()
# surv = survey.KanoSurvey()
# surv = survey.buildFairnessPrompts()
# surv = survey.DictatorGameSurvey()
surv.questions[:2]#.head()

Using default Personality Survey CSV file: surveys/survey_personality-test.csv


0    I am the life of the party.
1            I don't talk a lot.
Name: question, dtype: object

## Embedding

### Chunking

In [92]:
# PARAMETERS
EMBED_MODEL = "nomic-embed-text"        # nomic-embed-text = long ctx / mxbai-embed-large = big
CHUNK_SIZE = 30                         # Number of messages per chunk
OVERLAP_SIZE = 10                       # Number of overlapping messages between consecutive chunks
# COMMENT 04-16, perhaps we could try 5x retrievals with isolated semantics

In [93]:
# Initialize lists for storing chunks – and embeddings later
# different chunk size
chunks = []
stat_total_msgs_in_chunks = 0 # for statistics

# different chunk size
# Iterate over chats and messages to create chunks
for chat in et.selectedChats.values():
    messages = list(chat)  # Convert chat iterator to list for easier slicing
    num_messages = len(messages)

    # Create overlapping chunks of messages
    for i in range(0, num_messages - CHUNK_SIZE + 1, CHUNK_SIZE - OVERLAP_SIZE):
        chunk = messages[i:i + CHUNK_SIZE]  # Extract chunk of messages
        chunk_text = "\n".join(str(msg) for msg in chunk)  # Concatenate messages into a single string
        chunks.append(chunk_text)  # Append chunk to list of chunks

        stat_total_msgs_in_chunks += len(chunk) # For statistics

##### Display Info
total_messages = sum(len(chat) for chat in et.selectedChats.values())
chunks_count = len(chunks)
avg_chunk_char_len = np.mean([len(chunk) for chunk in chunks])

print(
    f"Chunk count: {chunks_count}",
    f"Average chunk character length: {round( avg_chunk_char_len)}",
    f"Rough estimate of tokens per chunk: {round(avg_chunk_char_len / 4)} (4 characters per token)",
    f"Messagees in input count: {total_messages}",
    f"Messages in chunks count: {stat_total_msgs_in_chunks}",
    f"Chunk \ Input ratio: {round(stat_total_msgs_in_chunks / total_messages,2)} (OVERLAP_SIZE={OVERLAP_SIZE})",
    f"Chunk Python type: {type(chunks[0])}",
    sep="\n"
) 

Chunk count: 130
Average chunk character length: 1244
Rough estimate of tokens per chunk: 311 (4 characters per token)
Messagees in input count: 2828
Messages in chunks count: 3900
Chunk \ Input ratio: 1.38 (OVERLAP_SIZE=10)
Chunk Python type: <class 'str'>


### Generaterating embeddings

In [95]:
########### Serialization ########
EMBEDDING_NAMEID = "mixtral"
AUTO_INFO = {
    "model": EMBED_MODEL,
    "CHUNK_SIZE": CHUNK_SIZE,
    "OVERLAP_SIZE": OVERLAP_SIZE,
    "chunks_count": chunks_count,
    "total_messages": total_messages,
    "stat_total_msgs_in_chunks": stat_total_msgs_in_chunks,
    "modules_chat": token_counts,
}
##################################

In [96]:
# Generate embeddings for each chunk
embeddings = []

progress, chunks_len = 0, len(chunks) # for progress bar
for chunk_text in chunks:
    progress += 1
    print(f"\rChunk {progress}/{chunks_len}", end="")

    embedding = ollama.embeddings(model=EMBED_MODEL, prompt=chunk_text)["embedding"]
    embeddings.append(embedding)

####################################################
# Generate embeddings for each chunk
# for chunk_text in chunks:
#     embedding = ollama.embeddings(model=EMBED_MODEL, prompt=chunk_text)["embedding"]
#     embeddings.append(embedding)


# token counts in all similar chunks
# tokens_in_chunks = 0
# for chunk in chunks_most_similar:
#     tokens_in_chunks += utils.count_tokens(chunk)
# print(f"Tokens in chunks: {tokens_in_chunks}")

bu.if_dir_not_exist_make("embeddings")
bu.quickJSON(AUTO_INFO, f"embeddings/{EMBEDDING_NAMEID}_info.json")
bu.quickJSON({"chunks": chunks, "embeddings": embeddings}, f"embeddings/{EMBEDDING_NAMEID}_embeddings.json")


Chunk 130/130

## Retrieval

In [97]:
subject = "elias"

# persona_small = "{small module}"
# persona_med = "{med module}"
# persona_text = "Favorite video games are Rimworld, Minecraft, Age of Empires, 7 Days to Die"

# Change below accoring to survey above
RETRIEVAL_PROMPT = "openess conciousness extrovert aggreableness neuroticism" #"personality"
# RETRIEVAL_PROMPT = "video game features"
# q_retrival_prompt =
# SURVEY_PROMPT = "Determine how much {subject} aggree with the statement. Guestimate how {subject} would answer to the question"

CHUNKS_COUNT_IN_CTX = 30 # Number of nearby chunks to put in context window

### Static

In [98]:
prompt_embedding = ollama.embeddings(model=EMBED_MODEL, prompt=RETRIEVAL_PROMPT)["embedding"]
chunks_most_similar_embeddings  = utils.find_most_similar(prompt_embedding, embeddings)[:CHUNKS_COUNT_IN_CTX]
chunks_most_similar = []
for embedding in chunks_most_similar_embeddings:
    chunks_most_similar.append(chunks[embedding[1]])

# Display results
bu.quickTXT("\n\n".join(chunks_most_similar), filename="ignorefolder/chunks.txt")

# token counts in all similar chunks
tokens_in_chunks = 0
for chunk in chunks_most_similar:
    tokens_in_chunks += utils.count_tokens(chunk)
print(f"Tokens in chunks: {tokens_in_chunks}")
####################################################
print(f"Chunks:{len(chunks)}, embeds:{len(embeddings)}")

Tokens in chunks: 10444
Chunks:130, embeds:130


### Dynamic

In [27]:
dynamic_retrieval_prompts = list(surv.questions)

CHUNKS_COUNT_IN_CTX = 5 # Number of nearby chunks to put in context window
dynamic_chunks_most_similar: List[List[str]] = [] 

progress = 0
lenn = len(dynamic_retrieval_prompts)
for prompt in dynamic_retrieval_prompts:
    progress += 1
    print(f"\rPrompt {progress}/{lenn}", end="")

    prompt_embedding = ollama.embeddings(model=EMBED_MODEL, prompt=prompt)["embedding"]
    chunks_most_similar_embeddings = utils.find_most_similar(prompt_embedding, embeddings)[:CHUNKS_COUNT_IN_CTX]
    chunks_most_similar = []
    for embedding in chunks_most_similar_embeddings:
        chunks_most_similar.append(chunks[embedding[1]])

    dynamic_chunks_most_similar.append(chunks_most_similar)
print(end="\n")
    
# VANITY PRINT
tokens_in_chunks = 0
for chunks_most_similar in dynamic_chunks_most_similar:
    for chunk in chunks_most_similar:
        tokens_in_chunks += utils.count_tokens(chunk)

del chunks_most_similar_embeddings # free memory
print(f"Tokens in average chunk group: {tokens_in_chunks/len(dynamic_chunks_most_similar)}")
bu.quickJSON(dynamic_chunks_most_similar, filename=f"ignorefolder/dynamic-chunks.json")

Prompt 50/50
Tokens in average chunk group: 1651.78


#### Vanity preview

In [19]:
preview_text = ""
PREVIEW_LIMIT = 10

for i, chunks_most_similar in enumerate(dynamic_chunks_most_similar):
    preview_text += f"==============Prompt: {dynamic_retrieval_prompts[i]}==============\n"
    for j, chunk in enumerate(chunks_most_similar):
        if j >= PREVIEW_LIMIT:
            break
        preview_text += f"=======CHUNK {j}=======\n{chunk}\n\n"
    preview_text += "\n\n"
bu.quickTXT(preview_text, filename=f"ignorefolder/dynamic-chunks_preview.txt")

## Prompt Builder

### With persona (dynamic)

In [28]:
final_prompts = []

for question, chunks_most_similar in zip(surv.questions, dynamic_chunks_most_similar):
    p = [
        systemMsg(
            "You are specialized in impersonating people. You will be presented to the subject through explicit datapoints of their digital footprint. In addition, you will deduct their implicit tastes by shadowing chats between the subject and friends. You will be asked to answer questions from the point of view of the persona. Text below:",
            "Conversations between persona and friends",
            "\nNEW CONVERSATION:\n".join(chunks_most_similar)
        ),
        # Understanding affirmation
        assistantMsg('I will answer from the point of view of the persona, based on what I could the deduct from the text provided.'),
        # Survey question. With Simulation
        userMsg("\n".join([
            f"Persona is surveyed about their video game survey. The persona must choose answer the question below with one of the given options: {', '.join(surv.POSSIBLE_ANSWERS)}. Persona's answer must only contain the chosen option, without any elaboration, nor introduction. ",
            question,
            "Persona chooses: "
        ])),
        # assistantMsg("\n".join([f"response: "
        # ])),
    ]
    final_prompts.append(p)

prompt_info = utils.describe_prompts_and_print(final_prompts)
bu.quickJSON(final_prompts, "ignorefolder/prompts.json")

Created 50 prompts.
Average prompt size: 1844 tokens.
Min prompt size: 1441, Max prompt size: 2648


### With Persona

In [99]:
PROMPT = {
    "role": "system", 
    "content": "You are an actor specializing in impersonating non-famouns people. You will be presented to the subject through explicit datapoints of their digital footprint. In addition, you will deduct their implicit personality traits by shadowing chats between the subject and friends. You will be asked to answer questions from the point of view of the persona. The persona you will be impersonating is named Elias. Context:"
    }

In [100]:
final_prompts = []

for question in surv.questions:
    p = [
        systemMsg(PROMPT['content']+"\n## chat conversions between subject and friends\n".join(chunks_most_similar)),
        assistantMsg('Understood. I will answer from the point of view of the persona, {subject}, based on what I could the deduct from the text provided above.'),
        userMsg("\n".join([
            f'\n\n**Your answer should only contain the chosen option without further explanation!** Reply to the statement below - how {subject} would reply - with one of these five options: {", ".join(surv.POSSIBLE_ANSWERS)}.',
            question,
            "The persona chooses: "
        ])),
    ]
    final_prompts.append(p)

print(f"{len(final_prompts)}")#,{final_prompts[:1]}")
prompt_info = utils.describe_prompts_and_print(final_prompts) # Vanity print
bu.quickJSON(final_prompts, "ignorefolder/prompts.json")

50
Created 50 prompts.
Average prompt size: 10878 tokens.
Min prompt size: 10875, Max prompt size: 10883


### Base (no persona)

In [ ]:
final_prompts = []

for question in surv.questions:
    p = [
        systemMsg(
            "You are participating in a survey. You will be presented with a series of questions about your video game preferrences.",
            f"You must choose answer to the question below with one of the five options: {', '.join(surv.POSSIBLE_ANSWERS)}. The answer must only contain the chosen option. "
        ),
        # Understanding affirmation
        assistantMsg('Understood. I will answer the question below with one of the given options.'),
        # Survey question. With Simulation
        userMsg(
            question,
            "Your choice: "
        ),
    ]
    final_prompts.append(p)

prompt_info = utils.describe_prompts_and_print(final_prompts) # Vanity print
bu.quickJSON(final_prompts, "ignorefolder/prompts.json")

# Run simulation

In [138]:
# load json file to dict
# with open("simulations/toElias/run2-airidas-personality_cv1_prompts.json", "r") as read_file:
# with open("simulations/toElias/run2-airidas-video-game-cv1_prompts.json", "r") as read_file:
# with open("simulations/toElias/run2-base-personality-cv1_prompts.json", "r") as read_file:
with open("simulations/toElias/run2-base-video-game-cv1_prompts.json", "r") as read_file:
    pre_final_prompts = json.load(read_file)


# pre_final_prompts

In [139]:
# Run Simulation
SETTINGS = {
     "model": "command-r-plus:104b-q2_K", # mixtral, command-r-plus:104b-q2_K
     # "temperature": 0.5,
     # best wizard and mixtral try mixtral-8x22b wizard in uCloud
}

##################################
SIM_ID = f"run2-base-video-game_rplus_cv2"
LIMIT = None # For testing purposes. Set to NONE to run all
AUTO_INFO = {
    "date": bu.get_timestamp(),
    # "EMBEDDING_NAMEID": EMBEDDING_NAMEID,
    # "RETRIEVAL_PROMPT": RETRIEVAL_PROMPT,
    # "CHUNKS_COUNT_IN_CTX": CHUNKS_COUNT_IN_CTX,
    # "survey_type": str(type(surv)),
    # "prompt_count": min(len(final_prompts), LIMIT) if LIMIT != None else len(final_prompts),
    # "avg_tokens_in_prompt": round(prompt_info["total_all_prompt_tokens"]/len(final_prompts)),
}

client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused
)

save = f"{SETTINGS['model']}_{SIM_ID}"
##################################

In [140]:
### ==== THE FUNCTIONAL 1!!!! =====
completions = []
l = len(final_prompts)
timer = bu.Benchmarker()
for i, (prompt, question) in enumerate(list(zip(final_prompts, surv.questions))):
    if LIMIT != None and i > LIMIT:
        break
    timer.mark()
    print(f"{i}/{l}...", end="\t") # Print progress
    # Send the Request    
    full_response = client.chat.completions.create(
        model=SETTINGS["model"],
        messages=prompt,
        # timeout=120,
        # temperature=SETTINGS["temperature"],
    )

    r = full_response.choices[0].message.content
    completions.append({'question': question, 'answer': r})
    print(f"{question}: {r}")

timer.mark()
# Save results
df = pd.DataFrame(completions)
# df.to_csv(f"results/{save}_simulation.csv", index=False)
df.to_csv(f"simulations/{SIM_ID}_simulation.csv", index=False)
# bu.quickJSON(final_prompts, f"results/{save}_prompts.json")
bu.quickJSON(final_prompts, f"ignorefolder/{SIM_ID}_prompts.json")
# bu.quickJSON(SETTINGS, f"results/{save}_setings.json")
bu.quickJSON({"settings": SETTINGS, "info": AUTO_INFO}, f"simulations/{SIM_ID}_info.json")

0/50...	I am the life of the party.: NEUTRAL
process_1 took 2.562s
1/50...	I don't talk a lot.: DISAGREE
2/50...	I feel comfortable around people.: NEUTRAL
  
Most likely response(s) by Elias to the statement '**I feel comfortable around people**: **
process_2 took 7.749s
3/50...	I keep in the background.: DISAGREE
4/50...	I start conversations.: NEUTRAL
process_3 took 1.678s
5/50...	I have little to say.: DISAGREE
6/50...	I talk to a lot of different people at parties.: NEUTRAL
process_4 took 1.683s
7/50...	I don't like to draw attention to myself.: SOMEWHAT DISAGREE
8/50...	I don't mind being the center of attention.: SOMEWHAT AGREE
process_5 took 1.954s
9/50...	I am quiet around strangers.: NEUTRAL
10/50...	I get stressed out easily.: DISAGREE
process_6 took 1.677s
11/50...	I am relaxed most of the time.: SOMEWHAT AGree
12/50...	I worry about things.: NEUTRAL
process_7 took 1.673s
13/50...	I seldom feel blue.: DISAGREE
14/50...	I am easily disturbed.: NEUTRAL
process_8 took 1.675s
1

#### dbug

In [36]:
df = pd.DataFrame(completions)

# Analysis

## Load

In [98]:
import pandas as pd
import json
SIMULATION_NAMEID = "elias-personality_rplus_cv1"
# SIM_ID

df = pd.read_csv(f'simulations/local/{SIMULATION_NAMEID}_simulation.csv')
# df = df.drop(df.columns[0], axis=1) #if loaded from csv, drop the added index col
with open(f'simulations/local/{SIMULATION_NAMEID}_info.json', 'r') as f:
    loaded = json.load(f)
try:
    AUTO_INFO = loaded["info"]
    SETTINGS = loaded["settings"]
    print("Settings and info loaded:")
    for k, v in AUTO_INFO.items():
        print(f"{k}: {v}")
    for k, v in SETTINGS.items():
        print(f"{k}: {v}")
except:
    print("No settings and/or info found")

try:
    if str(type(surv)) != AUTO_INFO["survey_type"]:
        print(f"WARNING: surv variable is not of the same type. {str(type(surv))} != {AUTO_INFO['survey_type']}")
except:
    pass

df.head(n=5)

Settings and info loaded:
date: 2024-04-18_204114
model: command-r-plus:104b-q2_K


,question,answer
0,I am the life of the party.,DISAGREE
1,I don't talk a lot.,DISAGREE
2,I feel comfortable around people.,SOMEWHAT AGREEE
3,I keep in the background.,NEUTRAL
4,I start conversations.,SOMEWHAT AGREEE


In [58]:
bu.if_dir_not_exist_make("simulations/results")
res = bu.LiveCSV(f"simulations/elias_runs.csv")

LiveCSV: File /Users/e/Documents/GitHub/Thesis/simulations/elias_runs.csv not existing. Creating new.
brikasutils.quickCSV: Saved 0 as simulations/elias_runs.csv


#### Proces simulation output - KANO

In [76]:
len(surv.test_answers["elias"])

40

In [70]:
surv = survey.KanoSurvey()
# Add airidas and elias answers
air = surv.test_answers["airidas"]
eli = surv.test_answers["elias"]
df.insert(2, "airidas", air[:len(df)])
df.insert(3, "elias", eli[:len(df)])

df['answer'] = df['answer'].str.upper()
df['airidas'] = df['airidas'].str.upper()
df['elias'] = df['elias'].str.upper()

Using default Kano Survey CSV file: surveys/survey_kano-model.csv


ValueError: Length of values (40) does not match length of index (50)

#### Proces simulation output - PERSONALITY

In [99]:
surv = survey.PersonalitySurvey()
# df = df.dropna()

# Add airidas and elias answers
air = surv.test_answers["airidas"]
eli = surv.test_answers["elias"]
df.insert(2, "airidas", air[:len(df)])
df.insert(3, "elias", eli[:len(df)])
df[:5]

Using default Personality Survey CSV file: surveys/survey_personality-test.csv


,question,answer,airidas,elias
0,I am the life of the party.,DISAGREE,3,3
1,I don't talk a lot.,DISAGREE,2,2
2,I feel comfortable around people.,SOMEWHAT AGREEE,4,4
3,I keep in the background.,NEUTRAL,2,4
4,I start conversations.,SOMEWHAT AGREEE,4,3


In [61]:
# compute one number of how the percentage of correct answers
result_data = {
    "Exact Matches": (df['answer'] == df['airidas']).sum() / len(df),
    "Correlation": df['answer'].corr(df['airidas']),
    "Exact Matches - elias": (df['answer'] == df['elias']).sum() / len(df),
    "Correlation - elias": df['answer'].corr(df['elias']),
}

for k, v in result_data.items():
    print(f"{k}: {v}")

ValueError: could not convert string to float: ' SOMEWHAT DISAGREE'

In [101]:
str(type(surv))

"<class 'survey.PersonalitySurvey'>"

In [100]:
new_res = {
    # "label": None,
    "SIMULATION_NAMEID": SIMULATION_NAMEID,#SIM_ID,
    "timestamp": bu.get_timestamp(),
    "survey_type": str(type(surv)),
    # "temperature": SETTINGS["temperature"],
    # "note": "",
    "exact_matches": result_data["Exact Matches"],
    "corr": result_data["Correlation"],
    "exact_matches_elias": result_data["Exact Matches - elias"],
    "corr_elias": result_data["Correlation - elias"],
}

tmp = bu.convert_dicts_to_table([new_res])
res.append_data(tmp[1], tmp[0])

brikasutils.quickCSV: Saved 8 as simulations/elias_runs.csv


### Cleaning

In [46]:
# remove all characters from a black list from the column answer
df['answer'] = df['answer'].apply(lambda x: x.strip())
for substr in utils.BLACKLIST_ANSWER_SUBSTRINGS:
    df['answer'] = df['answer'].apply(lambda x: re.sub(substr, "", x))
df['answer'] = df['answer'].str.upper()

df[30:]

,question,answer,airidas,elias
30,I am always prepared.,SOMEWHAT AGREE,4,1
31,I leave my belongings around.,SOMEWHAT DISAGREE,2,5
32,I pay attention to details.,AGREE,2,3
33,I make a mess of things.,SOMEWHAT DISAGREE,2,4
34,I get chores done right away.,SOMEWHAT DISAGREE,1,2
35,I often forget to put things back in their pro...,SOMEWHAT DISAGREE,2,2
36,I like order.,SOMEWHAT AGREE,4,4
37,I shirk my duties.,DISAGREE,2,1
38,I follow a schedule.,NEUTRAL,4,2
39,I am exacting in my work.,NEUTRAL,3,5


In [ ]:
# remove all characters from a black list from the column answer
df['answer'] = df['answer'].apply(lambda x: x.strip())

for substr in utils.BLACKLIST_ANSWER_SUBSTRINGS:
    df['answer'] = df['answer'].apply(lambda x: re.sub(substr, "", x))

df['answer'] = df['answer'].str.upper()
# Update isValid
df['isValid'] = df['answer'].apply(lambda x: x in surv.POSSIBLE_ANSWERS)

# if all values in isValid is true, drop the column, else print a message
if df['isValid'].all():
    df = df.drop('isValid', axis=1)
    print("All answers were valid")
else:
    print("Some answers were not valid")

df.head(n=10)

########################

# remove all characters from a black list from the column answer
for substr in utils.BLACKLIST_ANSWER_SUBSTRINGS:
    df['answer'] = df['answer'].apply(lambda x: re.sub(substr, "", x))

df['answer'] = df['answer'].str.upper()
# Update isValid
df['isValid'] = df['answer'].apply(lambda x: x in surv.POSSIBLE_ANSWERS)

# if all values in isValid is true, drop the column, else print a message
if df['isValid'].all():
    df = df.drop('isValid', axis=1)
    print("All answers were valid")
else:
    print("Some answers were not valid")

df
#### Cleanup
# remove all characters from a black list from the column answer
# for substr in utils.BLACKLIST_ANSWER_SUBSTRINGS:
#      df['answer'] = df['answer'].apply(lambda x: re.sub(substr, "", x))
# # Update isValid
#      df['isValid'] = df['answer'].apply(lambda x: x in surv.POSSIBLE_ANSWERS)

# if all values in isValid is true, drop the column, else print a message
# if df['isValid'].all():
#     df = df.drop('isValid', axis=1)
# else:
#     print("Some answers were not valid")

In [ ]:
df['answer'] = df['answer'].str.upper()
df['airidas'] = df['airidas'].str.upper()
df['elias'] = df['elias'].str.upper()

df['answer'] = df['answer'].map(remap_dict)
df['airidas'] = df['airidas'].map(remap_dict)
df['elias'] = df['elias'].map(remap_dict)

# df['CLONE_eli'] = df['CLONE_eli'].map(remap_dict)

# df = df.drop(columns=['uppercase_text'])
# df['CLONE_eli'] = df['answer'].apply(extract_uppercase_text)
# df['CLONE_eli'] = df['CLONE_eli'].str.upper()
# .str.upper() or .lower()
# df['answer'] = df['answer'].map(remap_dict, na_action='ignore')
#df['CLONE_eli'] = df['CLONE_eli'].fillna(0).astype(int)

### Remapping

In [47]:
if isinstance(surv, survey.KanoSurvey):
    remap_dict = {"I EXPECT IT": 5, "I LIKE IT": 4, "I AM NEUTRAL": 3, "I CAN TOLERATE IT": 2, "I DISLIKE IT": 1}
    df['answer'] = df['answer'].map(remap_dict)
    df['airidas'] = df['airidas'].map(remap_dict)
    df['elias'] = df['elias'].map(remap_dict)
elif isinstance(surv, survey.PersonalitySurvey):
    remap_dict = {"AGREE": 5, "SOMEWHAT AGREE": 4, "NEUTRAL": 3, "SOMEWHAT DISAGREE": 2, "DISAGREE": 1}
    df['answer'] = df['answer'].map(remap_dict)

### DF stats

### Remaps - UNIVERSAL

In [ ]:
#     pattern = r'\b(?:' + '|'.join(re.escape(phrase) for phrase in phrases_to_extract) + r')\b'
#     matches = re.findall(pattern, text, flags=re.IGNORECASE) 
#     return ' '.join(matches) if matches else ''

In [ ]:
# compute one number of how the percentage of correct answers
print(f"Exact Matches: {(df['CLONE_eli'] == df['IRL_eli']).sum() / len(df)}")
print(f"Correlation: {df['CLONE_eli'].corr(df['IRL_eli'])}")

df['elias_correct'] = df['CLONE_eli'] == df['IRL_eli']

## **Batch Sim**


In [ ]:
import importlib
import pandas as pd
from openai import OpenAI
import os
import json
import traceback
import sys

import brikasutils as bu
import shared_utils as utils
import survey
importlib.reload(bu)
importlib.reload(utils)
importlib.reload(survey)

queue = bu.FileRunQueue(queue_folder_path="batch/queue", completed_folder_path="batch/done")
report_live_csv = bu.LiveCSV("batch/run_reports.csv")
timer = bu.Benchmarker()


for filepath in queue:
    timer.mark_start(filepath)

    try: 
        ########## Handle batch stuff ########
        filename = os.path.splitext(os.path.basename(filepath))[0]
        with open(filepath, 'r') as f:
            rundata = json.load(f)

        # Load prompt file
        with open(rundata["instructions"]["prompt_file"], 'r') as f:
            final_prompts = json.load(f)

        # Make the surv
        if rundata["instructions"]["survey_type"] == "KanoSurvey":
            surv = survey.KanoSurvey()
        elif rundata["instructions"]["survey_type"] == "PersonalitySurvey":
            surv = survey.PersonalitySurvey()
        else:
            raise Exception("Invalid survey type")

        timestamp = bu.get_timestamp()
        ######### Run Simulation ########
        SIMULATION_NAMEID = filename
        LIMIT = rundata["instructions"]["LIMIT"] if "LIMIT" in rundata["instructions"] else None
        AUTO_INFO = {
            "date": timestamp,
            **rundata["info"], # unpacked from rundata
            "limit": LIMIT,
            "prompt_count": min(len(final_prompts), LIMIT) if LIMIT != None else len(final_prompts),
            "avg_tokens_in_prompt": round(utils.describe_prompts(final_prompts)["total_all_prompt_tokens"]/len(final_prompts)),
        }
        SETTINGS = {
            **rundata["settings"], # unpacked from rundata
        }

        # client depends on if it's local or not
        if rundata["instructions"]["isLocal"]:
            client = OpenAI(
                base_url = 'http://localhost:11434/v1',
                api_key='ollama', # required, but unused
            )
        else:
            client = OpenAI(
                api_key=os.environ.get("OPENAI_API_KEY"),
            )

        completions = []
        l = len(final_prompts)

        for i, (prompt, question) in enumerate(list(zip(final_prompts, surv.questions))):
            if LIMIT != None and i > LIMIT:
                break

            print(f"{i}/{l}...", end="\t") # Print progress
            # Send the Request
            full_response = client.chat.completions.create(
                messages=prompt,
                **SETTINGS,
            )
            r = full_response.choices[0].message.content

            completions.append({'question': question, 'answer': r})

            print(f"{question}: {r}")
            
        ############ Save Important results
        df = pd.DataFrame(completions)
        df.to_csv(f"batch/output/{SIMULATION_NAMEID}_simulation.csv", index=False)
        bu.if_dir_not_exist_make("batch/output/info")
        bu.quickJSON({"settings": SETTINGS, "info": AUTO_INFO}, f"batch/output/info/{SIMULATION_NAMEID}_info.json")

        status = "OK"
    
    except Exception:
        print(f"##### Error while running {filename}.")
        error_string = traceback.format_exc()
        print(error_string)
        status = "Failed"

    ########### Time the run
    try:
        time_taken = timer.mark_end(filepath)
    except:
        print("Error while timing run: ")
        print(traceback.format_exc())
        time_taken = None

    ########### Report the run
    try:
        new_report = {
            "filename": filename,
            "timestamp": timestamp,
            "time_taken": time_taken,
            "status": status,
            **rundata["instructions"],
            "error": error_string if status == "Failed" else "",
        }

        tmp = bu.convert_dicts_to_table([new_report])
        report_live_csv.append_data(tmp[1], tmp[0])
    except Exception as e:
        print(f"Error while reporting: ")
        traceback.print_exc()

    print(f"Processed {filename}. Stauts: {status}")

timer.print_total_execution_time()

## Modelfile

Force short JSON answer

Add this to the end of your prompt:
> ```json

Add this to the "stop" sequence:
>```

The idea is to force the model to continue writing json markdown. And end the generation when it outputs "```" which ends the json markdown section.

----

Command-r-plus
TEMPLATE """{{ if .System }}<|START_OF_TURN_TOKEN|><|SYSTEM_TOKEN|>{{ .System }}<|END_OF_TURN_TOKEN|>{{ end }}{{ if .Prompt }}<|START_OF_TURN_TOKEN|><|USER_TOKEN|>{{ .Prompt }}<|END_OF_TURN_TOKEN|>{{ end }}<|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>{{ .Response }}<|END_OF_TURN_TOKEN|>"""
PARAMETER stop "<|START_OF_TURN_TOKEN|>"
PARAMETER stop "<|END_OF_TURN_TOKEN|>"


Mixtral
TEMPLATE """ [INST] {{ .System }} {{ .Prompt }} [/INST]"""
PARAMETER stop "[INST]"
PARAMETER stop "[/INST]"



TEMPLATE """ [INST] {{ .System }} {{ .Prompt }} ```json [/INST] """
PARAMETER stop "[INST]"
PARAMETER stop "[/INST]"
PARAMETER stop "```"

----



mixtral x22
TEMPLATE """[INST] {{ if .System }}{{ .System }} {{ end }}{{ .Prompt }} [/INST]"""
PARAMETER stop "[INST]"
PARAMETER stop "[/INST]"

wizard x22
TEMPLATE """{{ if .System }}{{ .System }} {{ end }}{{ if .Prompt }}USER: {{ .Prompt }} {{ end }}ASSISTANT: {{ .Response }}"""
SYSTEM """A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions."""
PARAMETER stop "USER:"
PARAMETER stop "ASSISTANT:"

nomic_embed
TEMPLATE """{{ .Prompt }}"""
PARAMETER num_ctx 8192


Mistral 7b
TEMPLATE """[INST] {{ .System }} {{ .Prompt }} [/INST]"""
PARAMETER stop "[INST]"
PARAMETER stop "[/INST]"

Mistral 7b-wizard
TEMPLATE """{{ if .System }}{{ .System }} {{ end }}{{ if .Prompt }}USER: {{ .Prompt }} {{ end }}ASSISTANT: {{ .Response }}"""
SYSTEM """A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions."""
PARAMETER stop "USER:"
PARAMETER stop "ASSISTANT:"